# Implement Logistic Regression from scratch

In this assignment, you will implement Logistic Regression with L1 regularization from scratch and predict the labels of the test data. You will then verify the correctness of the your implementation using multiple "grader" functions/cells (provided by us) which will match your implmentation.

The grader functions would help you validate the correctness of your code. 

Please submit the final Colab notebook in the classroom ONLY after you have verified your code using the grader functions/cells.


**NOTE: DO NOT change the "grader" functions or code snippets written by us.Please add your code in the suggested locations.**

Ethics Code:
1. You are welcome to read up online resources to implement the code. 
2. You can also discuss with your classmates on the implmentation over Slack.
3. But, the code you write and submit should be yours ONLY. Your code will be compared against other stduents' code and online code snippets to check for plagiarism. If your code is found to be plagiarised, you will be awarded zero-marks for all assignments, which have a 10% weightage in the final marks for this course.

In [1]:
# Code to mount google drive in case you are loading the data from your google drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

ModuleNotFoundError: No module named 'google.colab'

In [11]:
# Loading data from csv file
import pandas as pd
data_path = 'logistic_regression_assignment_data.csv'

df = pd.read_csv(data_path)
df

,category,text
0,0,worldcom boss left books alone former worldc...
1,1,tigers wary of farrell gamble leicester say ...
2,1,yeading face newcastle in fa cup premiership s...
3,1,henman hopes ended in dubai third seed tim hen...
4,1,wilkinson fit to face edinburgh england captai...
...,...,...
1012,0,wall street cool to ebay s profit shares in on...
1013,0,ban on forced retirement under 65 employers wi...
1014,1,time to get tough on friendlies for an intern...
1015,0,christmas shoppers flock to tills shops all ov...


#### **Note:** Here class-0 is of category "business" and class-1 is of category "sport"

In [12]:
# Data Overiview
df['category'].value_counts()

1    509
0    508
Name: category, dtype: int64

### Creating Train and Test Datasets


In [13]:
# Splitting the data into train and test

from sklearn.model_selection import train_test_split

text = df['text']
category = df['category']
train_text, test_text, train_category, test_category = train_test_split(text, category, random_state=42, stratify=category, test_size=0.01)

print("Shape of Train_Text = ", train_text.shape)
print("Shape of Test_Text = ", test_text.shape)
print("Shape of Train_Category = ", train_category.shape)
print("Shape of Train_Category = ", test_category.shape)

Shape of Train_Text =  (1006,)
Shape of Test_Text =  (11,)
Shape of Train_Category =  (1006,)
Shape of Train_Category =  (11,)


## Custom Implementation

### Instructions:

  1. Read in the train_data.
  2. Vectorize train_data and test_data using sklearns built in tfidf vectorizer.
  3. Ignore unigrams and make use of both **bigrams & trigrams** and also limit the **max features** to **2000** and **minimum document frequency** to **10**.
  4. After the tfidf vectors are generated as mentioned above, next task is to column standardize your data.
  5. We want you to write in comments in your code, the reason you think for standardizing the data in the above step.
  6. You can use sklearn StandardScaler to column standardize your data.
  7. Write a function to initialise your weights & bias. And then run its corresponding grader function.
  8. Write a custom function to calculate sigmoid of a value. And then run its corresponding grader function to cross check your implementation of sigmoid function.
  9. Write a custom function to compute the total loss as the sum of log loss and l1 regularization loss based on true labels and predicted labels and weights. And you can crosscheck your implementation with its corresponding grader.
  10. Write a function to compute gradients for your weights and bias terms, which you have to make use of in updating your weights and bias while training your model.
  11. Implement a custom train function of logistic regression, wherein you take in the following inputs:
        * **X_train** which will be your vectorized text data
        * **y_train** which are the labels for your train data
        * **alpha** = 0.0001 which is the regularization factor (λ) 
        * **eta0** = 0.0001 which will be the learning rate   
        * **tolerance** = 0.001
        
  12. In the custom train function you should make use of a custom SGD function to update the weights and bias terms for **each** of your inputs. 
  13. The custom SGD implemented in the above train function for updating the weights and bias terms should run for many epochs until the difference in loss between two consecutive epochs is less than tolerance.

  14. Here one epoch means a complete iteration of your entire train data.
  15. Your train function should return the follwing:
        * the number of epochs it took to complete the training
        * train loss for all epochs
        * the values for final weights and bias terms.
        
  16. Now run the grader function to check whether the weights and bias obtained from your custom implementation are close enough to that of sklearns implementation.
  17. Next write a custom predict function which takes in as input the weights and bias values that you computed in your train function, and also takes in the test standardized data as input to predict its labels.
  18. Now run the grader function to check the accuracy of your predictions.





###Import necessary libraries

In [14]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
import matplotlib.pyplot as plt
import math

### 1. Vectorize train data and test data using sklearn tf-idf in the below cell





In [15]:
'''vectorize train and test data using TF-IDF and store them in train_vectors and test_vectors respectively'''
#defining the required parameters
max_features=2000 #max features for tf-idf
ngram_min=2 #min n_gram allowed
ngram_max=3 # max n_gram allowed
min_df=10 #min_df value for tf-idf vectorizer

vectorizer = TfidfVectorizer(ngram_range=(ngram_min,ngram_max),max_features=max_features,min_df=min_df)
train_vectors = vectorizer.fit_transform(train_text)

test_vectors = vectorizer.transform(test_text)
train_vectors.shape, test_vectors.shape

#print(train_vectors)

((1006, 2000), (11, 2000))

###2. Column standardize the train and test data

What is the reasoning for column standardizing the data?


Answer: To avoid variance in data across features

In [16]:
'''column standardize the train and test data and store them in train_vectors_stand and test_vectors_stand'''

scaler = StandardScaler()

train_vectors_stand = scaler.fit_transform(train_vectors.toarray())
test_vectors_stand = scaler.fit_transform(test_vectors.toarray())
train_vectors_stand.shape, test_vectors_stand.shape
#print(train_vectors_stand)

((1006, 2000), (11, 2000))

### 3. Custom function to intialise your weights and bias terms

In [17]:
def initialize_weights_bias(dim):
    ''' In this function, we will initialize our weights and bias terms'''

    # Initialize the weights to zeros array of (dim) dimensions. Here dim will be the number of features of your tfidf vectorizer output.
    # You can initialize the weight terms with zeros.
    # Initialize bias term to zero
    # Write your code below.
    w = np.zeros(dim,)
    b = 0

    return w,b

### Grader Function - 1

In [18]:
# Grader function to check the initialization of your weights and bias terms.

def grader_weights_bias(w,b):
  assert((len(w)==2000) and b==0)
  return True

dim = 2000
w,b = initialize_weights_bias(dim)
grader_1 = grader_weights_bias(w,b)
print("Grader_1 Status : ", grader_1)

Grader_1 Status :  True


### 4. Custom function to calculate sigmoid of a value

In [19]:
def custom_sigmoid(z):
    ''' In this function, we will return sigmoid of z'''
    
    # Compute sigmoid(z) and return its value.
    # Write your code below.
    
    sigmoid = 1 / (1 + np.exp(-z))

    return sigmoid

### Grader Function - 2

In [20]:
# Grader function to check the implementaiton of sigmoid function

def grader_sigmoid(z):
  val = custom_sigmoid(z)
  assert(val==0.8807970779778823)
  return True

grader_2 = grader_sigmoid(2)
print("Grader_2 Status : ", grader_2)

Grader_2 Status :  True


### 5.  Custom function to compute loss function

$logloss = -1*\frac{1}{n}\Sigma_{for each Y_{true},Y_{pred}}(Y_{true}log10(Y_{pred})+(1-Y_{true})log10(1-Y_{pred}))$ 


$L1 loss = \Sigma_{for each w}(|w|)$

$total loss = logloss + alpha*L1loss$<br>
Where alphas is the regularization parameter

In [36]:
def custom_loss(y_true, y_pred, alpha, w):
    '''In this function, we will compute total loss which is [(logloss) + (alpha * L1regularization loss)] '''
    
    eps = 1e-15

    y_pred = np.clip(y_pred,eps,1-eps)
    log_loss = (-1/len(y_true))*(np.sum([ i*np.log10(j) + ( (1-i)*np.log10(1-j) ) for i,j in zip(y_true, y_pred)]))
    #l1_loss = sum([abs(weight) for weight in w])
    #total_loss = log_loss+(alpha*l1_loss)
    
    #y_pred = np.clip(y_pred,eps,1-eps)
    #log_loss = (-1/len(y_true))*(np.sum(y_true * np.log(y_pred)))
    l1_loss = sum([abs(weight) for weight in w])
    total_loss = log_loss+(alpha*l1_loss)
    
    # Write your code below.
    #log_loss = 0
    #l1_loss = 0
    #for i in y_true:
        
     #   log_loss += ((y_true[i] * math.log10(y_pred[i])) - ((1 - y_true[i]) * math.log10(1 - y_pred[i])))
        #print(log_loss)
        
    #l1_loss = np.sum(w)  
    #print(l1_loss)
    #total_loss = -log_loss/len(y_true) + (alpha * l1_loss)

    return total_loss

### Grader Function - 3

In [37]:
# Grader function to check the implementaiton of logloss

def grader_loss():
  true_values = [1,1,0,1,0]
  pred_values = [0.9,0.8,0.1,0.8,0.2]
  w= np.array([0.1]*10)
  alpha= 0.0001
  #print(w)
  loss = custom_loss(true_values, pred_values,alpha,w)
  #print(loss)
  #print(0.07644900402910389+0.0001*10*0.1)
  assert(loss==0.07644900402910389+0.0001*10*0.1)
  return True


grader_3 = grader_loss()
print("Grader_3 Status : ", grader_3)

Grader_3 Status :  True


### 6. Custom function to updated weights and bias terms

Use the below formula to compute gradient of your weight and bias terms <br>
Loss term Li for a single example is given as below: 
<br>
<br>

$Li= -(Y_{i}log10(𝝈_{i})-(1-Y_{i})log10(1-𝝈_{i}) + \frac{alpha}{N}(sum(|w|))
$ <br>
<br>
$Where: 𝝈_{i} = σ(w^{T} x_i+b) $ <br>
<br>
And: L1 regularization = $\frac{alpha}{N}(sum(|w|)) $ <br>
Alpha: It is the Regularization parameter <br>
N : number of training examples<br>
σ : sigmoid function <br>
<br>
<br>
<br>
$dLi/dw= -Y_{i}x_{i}(1-𝝈_{i}) + (1-Y_{i})x_{i}𝝈_{i} + \frac{alpha}{N} \frac{w + (1e-5)}{|w + (1e-5)|}  $<br>
NOTE THAT: 1e-5 used in numerator and denominator to avoid division error <br>

$dLi/db= -Y_{i}(1-𝝈_{i}) + (1-Y_{i})𝝈_{i}$<br>
<br>
<br>
Hence,<br>
$dLi/dw= dw = (𝝈_{i} -Y_{i})x_{i} + \frac{alpha}{N}\frac{w + (1e-5)}{|w + (1e-5)|} $<br>
1e-5 used in numerator and denominator to avoid division error <br>
$dLi/db =  db = 𝝈_{i}-Y_{i}$
<br>
<br>
!!NOTE: USE NEGATIVE GRADIENT WHILE UPDATING

### 6a. Custom function to compute Gradient of loss function wrt weights:

In [48]:
def gradient_dw(x, y, w, b, alpha, N):
    '''In this function, we will compute the gardient w.r.t. w '''
    #sigma_i = custom_sigmoid(w.dot(x.T) + b)
    #dw = ( (sigma_i - y).dot(x)) + ( ( alpha/N )*( np.sign(w) ) )
    #return dw
    
    # Write your code below.
    dotProd = w.dot(x.T) + b #(np.dot(w.T,x)+b)
    sigmoid = custom_sigmoid(dotProd)
    
    db = sigmoid - y
    #print(db)
    dw = db.dot(x) +( (alpha/N)*(np.sign(w))) #((w+1e-5)/abs(w+1e-5)))# (np.sign(w))
    
    return dw





### 6b.  Custom function to compute Gradient of loss function wrt bias term:

In [24]:
def gradient_db(x, y, w, b):
    '''In this function, we will compute the gardient w.r.t. b '''
    sigma_i = custom_sigmoid(w.dot(x.T) + b)
   
    db = custom_sigmoid(w.dot(x.T) + b) - y     
    return db

    # Write your code below.
    # 𝝈𝑖=σ(𝑤𝑇𝑥𝑖+𝑏), 𝝈𝑖−𝑌𝑖
    #dotProd = (np.dot(w.T,x)+b)
    #sigmoid = custom_sigmoid(dotProd)
    
    #db = sigmoid - y

    #return db

###6c. Custom function to train logistic regression model

$w^{(t+1)}← w^{(t)}- eta0*(dw^{(t)}) $<br>
$b^{(t+1)}←b^{(t)} - eta0*(db^{(t)}) $

In [45]:
def custom_train(X_train, y_train,alpha, eta0,tolerance):
  """
  In this function we will compute optimal values for weights and bias terms on
  the train data. 

  Here eta0 is the learning rate and alpha is the regularization term.
  """
  train_loss=[]
    
    
  num_epochs = 0
  continue_loop = True

  # Implement the code as follows:

  # 1. Initalize the weights (call the initialize_weights(X_train[0]) function)
  # 2. Repeat For many epochs until condition "e"  fails
          # a) for every data point(X_train,y_train)
                # compute gradient w.r.to w (call the gradient_dw() function)
                # compute gradient w.r.to b (call the gradient_db() function)
                # update w, b using the above eqns
          # b) predict the output of x_train[for all data points in X_train] using w,b
          # c) compute the loss between predicted and actual values (call the loss function)
          # d) store all the train loss values in a list
          # e) Compare previous loss and current loss, if the difference between loss is not more than or equal to the tolerance, stop the process and return w,b

  # 3. Return the values of weights, bias, train_loss and num_epochs
  N=len(X_train)
  w,b = initialize_weights_bias(X_train.shape[1])
  
  print("initial b",b)  
  print("initial w",w)
  #condition to run the training for more epochs
  cond=True

  #checking whether to run the training for more epochs
  while(cond):
    
    #looping through each datapoint
    for j in range(len(X_train)):
        x=X_train[j]
        y=y_train[j]
        
        #computing gradients
        dw=gradient_dw(x,y,w,b,alpha,N)
        db=gradient_db(x,y,w,b)
        
        #updating weights
        w=w-eta0*dw
        b=b-eta0*db

    #computing y_pred for entire train data as y_pred= sigmoid(wTx+b)
    y_pred_train= np.array([custom_sigmoid(np.dot(w,X_traini)+b) for X_traini in X_train])

    #adding the train loss of current epoch to the list
    train_loss.append(custom_loss(y_train,y_pred_train,alpha,w))

    #checking whether loss has improved from previous epoch by atleast tolerance
    if num_epochs>0:
      cond=((train_loss[num_epochs-1]-train_loss[num_epochs])>=tolerance)

    #increasing the number of epochs the model is trained.
    num_epochs=num_epochs+1
        
      #if num_epochs>0:
        #continue_loop=((train_loss[num_epochs-1]-train_loss[num_epochs])>tolerance)
    
      #num_epochs+=1 # increase number of epoch
      #print(train_loss)  
      #print(num_epochs)
      #print(b)
      #print(w)
      #eta0 = eta0*0.90
      
        
        
  b = np.mean(b)
                         
    
  
    
  return w,b,train_loss,num_epochs
  




### Grader Function - 4

In [49]:
def grader_weights_bias():
  # fitting sklearn SGD classifier
  clf = linear_model.SGDClassifier(eta0=0.0001, alpha=0.0001, loss='log', random_state=15, penalty='l1', tol=1e-3, learning_rate='constant')
  clf.fit(train_vectors_stand,train_category.values)
  model_coef= clf.coef_[0]

  # fitting custom train with same learning rate, regularization and tolerance as of sklearn
  w,b,_,epoch = custom_train(train_vectors_stand, train_category.values, 0.0001,0.0001,0.001)

  # checking whether the weights and bias returned by both the implementations are closer
  assert((not (w-model_coef>0.02).any())==True)
  assert(not (b-clf.intercept_>0.02)==True)
  
  return True

grader_4 = grader_weights_bias()
print("Grader_4 Status : ", grader_4)

initial b 0
initial w [0. 0. 0. ... 0. 0. 0.]


AttributeError: 'numpy.float64' object has no attribute 'dot'

### 7. Plot the train loss with x as epoch number and y as train loss

In [10]:
# plotting graph for epoch vs loss for train and test data

w,b,train_loss,epochs = custom_train(train_vectors_stand, train_category.values, 0.0001,0.0001,0.001)
plt.plot(range(epochs),train_loss,label='train curve')
plt.title('epoch vs loss')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

NameError: name 'train_vectors_stand' is not defined

### 8. Custom function to make predictions using logistic regression model

In [55]:
def predict(w,b, X):
    '''function to predict label given weights, bias and standardized data'''
    predictions = w.dot(X.T) + b
    predictions[predictions>=0] = 1
    predictions[predictions<0] = 0
    return predictions #it should be a numpy array

### Grader Function - 5

In [160]:
def grader_predict():
  ''' grader to check the test accuracy'''
  w,b,_,_ = custom_train(train_vectors_stand, train_category.values, 0.0001,0.0001,0.001)
  test_preds= predict(w,b,test_vectors_stand)
  test_accuracy= (np.sum(test_category==test_preds)/len(test_preds))*100
  if(test_accuracy>=90):
    print("Success!")
  else:
    print("Failed! \n Test accuracy = ", test_accuracy)
  return
  
grader_predict()

initial b 0
initial w [0. 0. 0. ... 0. 0. 0.]
Current loss: 0.361402963290776
Current loss: 0.3061373542545835
Current loss: 0.10568492122158166
Current loss: 0.09325677807196033
Current loss: 0.058553321000566615
Current loss: 0.05098133413952792
Current loss: 0.04753496721033458
Current loss: 0.04515559012677477
Current loss: 0.04335655563916297
Current loss: 0.041850541914598166
Current loss: 0.009750112991639637
Current loss: 0.007672689981236568
Current loss: 0.00644922657280109
Current loss: 0.005588079890274324
Success!


**Extension**

Learning rate need not be constant always. We can choose to vary the learning rate depending upon various criteria. One simple implementation is to decrease the learning rate by a constant percentage every epoch

In [163]:
def custom_train_v2(X_train, y_train,alpha, eta0, tolerance):
  """
  In this function we will compute optimal values for weights and bias terms on
  the train data. 

  Here eta0 is the learning rate and alpha is the regularization term.
  """

  #array to store train loss for each epoch
  train_loss=[]

  # Implement the code as follows:

  # 1. Initalize the weights (call the initialize_weights(X_train[0]) function)
  # 2. Repeat For many epochs until condition "e"  fails
          # a) for every data point(X_train,y_train)
                # compute gradient w.r.to w (call the gradient_dw() function)
                # compute gradient w.r.to b (call the gradient_db() function)
                # update w, b using the above eqns
          # b) predict the output of x_train[for all data points in X_train] using w,b
          # c) compute the loss between predicted and actual values (call the loss function)
          # d) store all the train loss values in a list
          # e) Compare previous loss and current loss, if the difference between loss is not more than or equal to the tolerance, stop the process and return w,b

  # 3. Return the values of weights, bias, train_loss and num_epochs 
  
  #initializing the weights and bias
  w,b=initialize_weights_bias(X_train.shape[1])

  #storing the number of train points in N
  N=len(X_train)

  #number of epochs the model is trained
  num_epochs=0

  #condition to run the training for more epochs
  cond=True

  #checking whether to run the training for more epochs
  while(cond):
    
    #looping through each datapoint
    for j in range(len(X_train)):
        x=X_train[j]
        y=y_train[j]
        
        #computing gradients
        dw=gradient_dw(x,y,w,b,alpha,N)
        db=gradient_db(x,y,w,b)
        
        #updating weights
        w=w-eta0*dw
        b=b-eta0*db

    #computing y_pred for entire train data as y_pred= sigmoid(wTx+b)
    y_pred_train= np.array([custom_sigmoid(np.dot(w,X_traini)+b) for X_traini in X_train])

    #adding the train loss of current epoch to the list
    train_loss.append(custom_loss(y_train,y_pred_train,alpha,w))

    #checking whether loss has improved from previous epoch by atleast tolerance
    if num_epochs>0:
      cond=((train_loss[num_epochs-1]-train_loss[num_epochs])>=tolerance)

    #increasing the number of epochs the model is trained.
    num_epochs=num_epochs+1

    #decreasing learning rate by 10% each epoch
    eta0 = eta0*0.90  

  return w,b,train_loss,num_epochs